# Percentile Method

In [38]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm

First, we read data from the log_normal file and store our 15 samples

In [39]:
def read_numbers_from_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        numbers = content.split()
        numbers = [float(num) for num in numbers]
    return numbers


file_path = 'logNormal.txt'
data = read_numbers_from_file(file_path)

Now, we create a funcn for bootstrapping from the sample and create the distribution of its mean

In [40]:
def bootstrap_means(data, n_bootstrap=1000):
    bootstrap_samples = np.random.choice(data, (n_bootstrap, len(data)), replace=True)
    bootstrap_means = np.mean(bootstrap_samples, axis=1)
    return bootstrap_means

Now we use percentile method, by cutting off X samples corresponding to aplha from right and from left

In [41]:
def percentile_confidence_interval(data, alpha=0.05):
    lower_percentile = 100 * (alpha / 2)
    upper_percentile = 100 * (1 - alpha / 2)
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return lower_bound, upper_bound

In [42]:
n_bootstrap = 1000
alpha = 0.05

bootstrap_means_sample = bootstrap_means(data, n_bootstrap)
lower_bound, upper_bound = percentile_confidence_interval(bootstrap_means_sample, alpha)
print(f"95% confidence interval from percentile: ({lower_bound}, {upper_bound})")

95% confidence interval from percentile: (1.3673430611833333, 3.8614479307833327)


### Plot the confidence interval

In [43]:
fig = px.histogram(bootstrap_means_sample, nbins=30, title='Bootstrap Means Histogram with 95% Confidence Interval Percentile')

fig.add_vline(x=lower_bound, line=dict(color='red', dash='dash'), annotation_text=f'Lower bound ({lower_bound:.2f})',
              annotation_position="top left")
fig.add_vline(x=upper_bound, line=dict(color='green', dash='dash'), annotation_text=f'Upper bound ({upper_bound:.2f})',
              annotation_position="top right")

fig.add_trace(go.Scatter(
    x=[lower_bound, upper_bound],
    y=[0, 0],
    mode='lines+text',
    text=['', '95% CI'],
    textposition='top center',
    line=dict(color='black', width=3)
))

fig.update_layout(
    xaxis_title='Mean Value',
    yaxis_title='Frequency',
    legend_title_text='Confidence Interval',
    title_x=0.5
)
fig.show()

# BCa (Bias-Corrected and accelerated)

In [44]:
def bca_confidence_interval(data, bootstrap_means_sample, alpha=0.05):
    theta_hat = np.mean(data)
    z0 = norm.ppf((np.sum(bootstrap_means_sample < theta_hat)) / len(bootstrap_means_sample))
    jackknife_means = np.array([np.mean(np.delete(data, i)) for i in range(len(data))])
    jackknife_mean = np.mean(jackknife_means)
    a = np.sum((jackknife_mean - jackknife_means) ** 3) / (6 * np.sum((jackknife_mean - jackknife_means) ** 2) ** 1.5)
    lower_percentile = 100 * norm.cdf(z0 + (z0 + norm.ppf(alpha / 2)) / (1 - a * (z0 + norm.ppf(alpha / 2))))
    upper_percentile = 100 * norm.cdf(z0 + (z0 + norm.ppf(1 - alpha / 2)) / (1 - a * (z0 + norm.ppf(1 - alpha / 2))))
    lower_bound = np.percentile(bootstrap_means_sample, lower_percentile)
    upper_bound = np.percentile(bootstrap_means_sample, upper_percentile)
    return lower_bound, upper_bound

Calculate the lower and upper intervals

In [45]:
lower_bound, upper_bound = bca_confidence_interval(data, bootstrap_means_sample, alpha)
print(f"95% BCa confidence interval: ({lower_bound}, {upper_bound})")

95% BCa confidence interval: (1.6073052918772743, 4.219976342660103)


### Plot the Interval

In [46]:
fig = px.histogram(bootstrap_means_sample, nbins=30, title='Bootstrap Means Histogram with 95% BCa Confidence Interval')

fig.add_vline(x=lower_bound, line=dict(color='red', dash='dash'), annotation_text=f'Lower bound ({lower_bound:.2f})',
              annotation_position="top left")
fig.add_vline(x=upper_bound, line=dict(color='green', dash='dash'), annotation_text=f'Upper bound ({upper_bound:.2f})',
              annotation_position="top right")

fig.add_trace(go.Scatter(
    x=[lower_bound, upper_bound],
    y=[0, 0],
    mode='lines+text',
    text=['', '95% CI'],
    textposition='top center',
    line=dict(color='black', width=3)
))

fig.update_layout(
    xaxis_title='Mean Value',
    yaxis_title='Frequency',
    legend_title_text='Confidence Interval',
    title_x=0.5
)

fig.show()

## Comparison between miss percentage of both the Methods

In [47]:
population = data
n_simulations = 10000
n_bootstrap = 1000
sample_size = 15
alpha = 0.05
true_mean = np.mean(population)
percentile_miss_left = 0
percentile_miss_right = 0
bca_miss_left = 0
bca_miss_right = 0

In [48]:
for _ in range(n_simulations):
    sample = np.random.choice(population, sample_size, replace=True)
    bootstrap_means_sample = bootstrap_means(sample, n_bootstrap)

    perc_lower, perc_upper = percentile_confidence_interval(bootstrap_means_sample, alpha)
    bca_lower, bca_upper = bca_confidence_interval(sample, bootstrap_means_sample, alpha)

    if perc_lower > true_mean:
        percentile_miss_left += 1
    elif perc_upper < true_mean:
        percentile_miss_right += 1

    if bca_lower > true_mean:
        bca_miss_left += 1
    elif bca_upper < true_mean:
        bca_miss_right += 1

percentile_miss_left /= n_simulations
percentile_miss_right /= n_simulations
bca_miss_left /= n_simulations
bca_miss_right /= n_simulations

In [49]:
bt_means = bootstrap_means(data, n_bootstrap)

alpha = 0.05
percentile_lower_bound, percentile_upper_bound = percentile_confidence_interval(bt_means, alpha)
bca_lower_bound, bca_upper_bound = bca_confidence_interval(data, bt_means, alpha)

# Print the confidence intervals
print(f"95% Percentile confidence interval: ({percentile_lower_bound}, {percentile_upper_bound})")
print(f"95% BCa confidence interval: ({bca_lower_bound}, {bca_upper_bound})")

95% Percentile confidence interval: (1.3519111825166668, 3.8310231509833326)
95% BCa confidence interval: (1.5793480496718217, 4.200697966516587)


In [50]:
# Visualization
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Percentile Miss Left', 'Percentile Miss Right', 'BCa Miss Left', 'BCa Miss Right'],
    y=[percentile_miss_left, percentile_miss_right, bca_miss_left, bca_miss_right],
    marker_color=['blue', 'blue', 'red', 'red']
))

fig.update_layout(
    title='Miss Percentages for Percentile and BCa Methods',
    xaxis_title='Miss Type',
    yaxis_title='Percentage',
    yaxis=dict(tickformat=".2%"),
)

fig.show()

print(f"Percentile Method Miss Left: {percentile_miss_left:.2%}")
print(f"Percentile Method Miss Right: {percentile_miss_right:.2%}")
print(f"BCa Method Miss Left: {bca_miss_left:.2%}")
print(f"BCa Method Miss Right: {bca_miss_right:.2%}")

Percentile Method Miss Left: 1.43%
Percentile Method Miss Right: 12.22%
BCa Method Miss Left: 2.63%
BCa Method Miss Right: 8.89%


# Conclusion

### For 95% the ideal left and right miss should be 2.5% 
### We can observe that BCa has a better miss left and miss right percentage compared to Percentile. 
### Hence BCa gives a better interval compared to Percentile method.